<a href="https://colab.research.google.com/github/Jefferson2325353/metodos-matematicos-fisica/blob/main/Ejemplo_5_1_con_SymPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Ejemplo 5.1 con SymPy

import sympy as sp
sp.init_printing()


def geom_from_mapping(x_vec, q_vec, t):
    """
    Dado un mapeo cartesiano x(q(t)) devuelve:
    - J: Jacobiano ∂x^i/∂q^{j'}
    - g: Métrica g_{j'k'} = J^T J
    - T: Energía cinética geométrica = 1/2 dq^T g dq
    - v_cov: v_{j'} = ∂T/∂(dq^{j'})
    - a_cov: a_{j'} = d/dt(∂T/∂(dq^{j'})) - ∂T/∂(q^{j'})
    """
    q = sp.Matrix(q_vec)
    dq = sp.diff(q, t)


    J = x_vec.jacobian(q)
    g = sp.simplify(J.T * J)

    # Energía cinética geométrica
    T = sp.simplify(sp.Rational(1, 2) * (dq.T * g * dq)[0])


    v_cov = sp.Matrix([sp.diff(T, dq[i]) for i in range(len(q))])
    dt = lambda expr: sp.diff(expr, t)
    a_cov = sp.Matrix([ sp.simplify(dt(sp.diff(T, dq[i])) - sp.diff(T, q[i]))
                        for i in range(len(q)) ])

    return J, g, T, sp.simplify(v_cov), sp.simplify(a_cov)


t = sp.symbols('t', real=True)

r = sp.Function('r')(t)
theta = sp.Function('theta')(t)
phi = sp.Function('phi')(t)

q_vec = [r, theta, phi]

x_vec = sp.Matrix([
    r*sp.sin(theta)*sp.cos(phi),
    r*sp.sin(theta)*sp.sin(phi),
    r*sp.cos(theta)
])

J, g, T, v_cov, a_cov = geom_from_mapping(x_vec, q_vec, t)


# Impresión de resultados

print("== Jacobiano J_{i j'} = ∂x^i/∂q^{j'} ==")
sp.pprint(J)

print("\n== Métrica g_{j'k'} = J^T J ==")
sp.pprint(g)

print("\n== T = 1/2 * dq^T * g * dq ==")
sp.pprint(T)

print("\n== Componentes covariantes de la velocidad v_{j'} = ∂T/∂\\dot q^{j'} ==")
sp.pprint(v_cov)

print("\n== Componentes covariantes de la aceleración a_{j'} = d/dt(∂T/∂\\dot q^{j'}) - ∂T/∂q^{j'} ==")
sp.pprint(a_cov)


# Comprobaciones conocidas en esféricas

rd, td, pd = sp.diff(r, t), sp.diff(theta, t), sp.diff(phi, t)
rdd, tdd, pdd = sp.diff(r, t, 2), sp.diff(theta, t, 2), sp.diff(phi, t, 2)

T_expected = sp.Rational(1,2)*( rd**2 + (r**2)*(td**2) + (r**2)*(sp.sin(theta)**2)*(pd**2) )
v_expected = sp.Matrix([ rd, r**2*td, r**2*(sp.sin(theta)**2)*pd ])

a_r_exp = rdd - r*(td**2) - r*(sp.sin(theta)**2)*(pd**2)
a_t_exp = r**2*tdd + 2*r*rd*td - r**2*sp.sin(theta)*sp.cos(theta)*(pd**2)
a_p_exp = r**2*(sp.sin(theta)**2)*pdd + 2*r*rd*(sp.sin(theta)**2)*pd + 2*r**2*sp.sin(theta)*sp.cos(theta)*td*pd
a_expected = sp.Matrix([a_r_exp, a_t_exp, a_p_exp])

print("\n== Verificación (esféricas) ==")
print("¿T coincide?            ", sp.simplify(T - T_expected) == 0)
print("¿v_cov coincide?        ", all(sp.simplify((v_cov - v_expected)[i])==0 for i in range(3)))
print("¿a_cov coincide?        ", all(sp.simplify((a_cov - a_expected)[i])==0 for i in range(3)))


== Jacobiano J_{i j'} = ∂x^i/∂q^{j'} ==
⎡sin(θ(t))⋅cos(φ(t))  r(t)⋅cos(φ(t))⋅cos(θ(t))  -r(t)⋅sin(φ(t))⋅sin(θ(t))⎤
⎢                                                                        ⎥
⎢sin(φ(t))⋅sin(θ(t))  r(t)⋅sin(φ(t))⋅cos(θ(t))  r(t)⋅sin(θ(t))⋅cos(φ(t)) ⎥
⎢                                                                        ⎥
⎣     cos(θ(t))           -r(t)⋅sin(θ(t))                   0            ⎦

== Métrica g_{j'k'} = J^T J ==
⎡1    0           0        ⎤
⎢                          ⎥
⎢    2                     ⎥
⎢0  r (t)         0        ⎥
⎢                          ⎥
⎢           2       2      ⎥
⎣0    0    r (t)⋅sin (θ(t))⎦

== T = 1/2 * dq^T * g * dq ==
                           2                   2             2
 2       2       ⎛d       ⎞     2    ⎛d       ⎞    ⎛d       ⎞ 
r (t)⋅sin (θ(t))⋅⎜──(φ(t))⎟    r (t)⋅⎜──(θ(t))⎟    ⎜──(r(t))⎟ 
                 ⎝dt      ⎠          ⎝dt      ⎠    ⎝dt      ⎠ 
──────────────────────────── + ───────────────── + ───────────
    